# Calculating the Score

In [3]:
import os
import json
import difflib
import psycopg2
from decimal import Decimal, ROUND_DOWN
from dotenv import load_dotenv

# ─── 0) Setup & load ───────────────────────────────────────────────────
load_dotenv(r"C:\Point Detection\.env")
DSN = os.getenv("DATABASE_URL")

# ─── 1) Read parsed summary ────────────────────────────────────────────
summary_path = r"C:\Point Detection\preprocessing\processed\parsed_summary.json"
with open(summary_path, encoding="utf-8") as f:
    summary = json.load(f)

store_name  = summary["store_name"]
total_price = Decimal(str(summary["total_price"]))

# ─── 2) Connect & load store_categories ───────────────────────────────
conn = psycopg2.connect(DSN)
cur  = conn.cursor()
cur.execute("SELECT store_name, bucket FROM store_categories;")
store_map  = dict(cur.fetchall())
store_keys = list(store_map.keys())

# ─── 3) Fuzzy-match (strict) ───────────────────────────────────────────
matches = difflib.get_close_matches(store_name, store_keys, n=1, cutoff=0.6)
if not matches:
    raise ValueError(f"No matching store for {store_name!r}")
best_store  = matches[0]
best_bucket = store_map[best_store]

# ─── 4) Compute points ────────────────────────────────────────────────
if best_bucket in ("零售", "餐饮"):
    points = int(total_price)
else:
    points = int((total_price / Decimal(5)).to_integral_value(rounding=ROUND_DOWN))

# ─── 5) Build output & write JSON ──────────────────────────────────────
result = {
    "store_name_raw": store_name,
    "matched_store":  best_store,
    "bucket":         best_bucket,
    "total_price":    float(total_price),
    "points":         points
}

out_path = r"C:\Point Detection\preprocessing\processed\receipt_points.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Written points info to {out_path}")
print(json.dumps(result, ensure_ascii=False, indent=2))

# ─── 6) Cleanup ───────────────────────────────────────────────────────
cur.close()
conn.close()


Written points info to C:\Point Detection\preprocessing\processed\receipt_points.json
{
  "store_name_raw": "Holiland",
  "matched_store": "Holiland Travel",
  "bucket": "餐饮",
  "total_price": 74.0,
  "points": 74
}
